In [2]:
!pip install -U openai-whisper -q
!sudo apt-get install ffmpeg -y -q
!pip install langchain langchain-groq "langchain-community>=0.3.30" chromadb beautifulsoup4 requests sentence-transformers playwright -q
!playwright install chromium
!pip install gradio langchain-google-genai -q
!pip install googlemaps -q
!pip install -U langchain-classic
!pip install gradio -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 46.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Reading package lists...
Building dependency tree...
Reading state information...
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 112.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 120.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import gradio as gr
import whisper
import os
import requests
from bs4 import BeautifulSoup
import nest_asyncio
import smtplib
import ssl
from email.message import EmailMessage
import asyncio
from langchain.tools import tool
from langchain_groq import ChatGroq
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langchain_classic.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.messages import HumanMessage, AIMessage
from playwright.async_api import async_playwright
import googlemaps
# Import the new Google Chat Model
from langchain_google_genai import ChatGoogleGenerativeAI
nest_asyncio.apply()

Model Initailization

In [5]:
llm = ChatGoogleGenerativeAI(model="models/gemini-2.5-flash", temperature=0.2)
model_name = "all-MiniLM-L6-v2"
embedding_function = HuggingFaceEmbeddings(model_name=model_name)
vectorstore = Chroma(collection_name="university_info", embedding_function=embedding_function)
print("✅ LLM, Embeddings, and Vector Store Initialized.")

print("🎤 Loading Whisper model...")
whisper_model = whisper.load_model("base")
print("✅ Whisper model loaded.")

/tmp/ipython-input-2409444575.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = HuggingFaceEmbeddings(model_name=model_name)
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/tmp/ipython-input-2409444575.py:4: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vectorstore = Chroma(collection_name="university_info", embedding_function=embedding_function)


✅ LLM, Embeddings, and Vector Store Initialized.
🎤 Loading Whisper model...


100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 251MiB/s]


✅ Whisper model loaded.


Tools

In [6]:
@tool
def send_email(recipient_email: str, subject: str, body: str):
    """
    Use this tool ONLY to send an email AFTER the user has approved a draft.
    You must provide all arguments: recipient_email, subject, and body.
    """
    print(f"⚙️ TOOL CALLED: send_email to {recipient_email}")
    msg = EmailMessage()
    msg.set_content(body)
    msg["Subject"] = subject
    msg["From"] = SENDER_EMAIL
    msg["To"] = recipient_email
    context = ssl.create_default_context()
    try:
        with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
            server.login(SENDER_EMAIL, SENDER_PASSWORD)
            server.send_message(msg)
        confirmation_message = f"✅ Email successfully sent to {recipient_email}."
    except Exception as e:
        confirmation_message = f"❌ Failed to send email. Error: {e}"
    print(confirmation_message)
    return confirmation_message

@tool
def draft_email(recipient_email: str, subject: str, body: str):
    """
    Use this tool to create a draft of an email and show it to the user for approval.
    You must provide all arguments: recipient_email, subject, and body.
    """
    print(f"⚙️ TOOL CALLED: draft_email for {recipient_email}")
    draft = (
        f"I have drafted the following email:\n\n"
        f"-------------------\n"
        f"To: {recipient_email}\n"
        f"Subject: {subject}\n\n"
        f"{body}\n"
        f"-------------------\n\n"
        f"Should I send this email? (Yes/No)"
    )
    return draft

@tool
def book_study_room(name: str, date: str, time: str, num_people: str):
    """
    Use this tool ONLY to book a university study room.
    You must provide all of the following arguments: the name for the booking,
    the date, the time, and the number of people.
    """
    print(f"⚙️ TOOL CALLED: book_study_room for {name}")

    async def fill_form_async():
        form_url = "https://docs.google.com/forms/d/e/1FAIpQLSc9HAhtD3shaYZza-k2ozpPPFWMKTW7JuH935z9-UqhUOVhbg/viewform"
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True)
            page = await browser.new_page()
            await page.goto(form_url)
            try:
                await page.get_by_label("Name").fill(name)
                await page.get_by_label("Date").fill(date)
                await page.get_by_label("Time").fill(time)
                await page.get_by_label("Number of people").fill(num_people)
                await page.get_by_role("button", name="Submit").click()
                await page.wait_for_load_state('networkidle')
                confirmation_message = f"✅ Successfully submitted the booking for {name}."
            except Exception as e:
                confirmation_message = f"❌ Error during form submission: {e}"
            finally:
                await browser.close()
        return confirmation_message

    result = asyncio.run(fill_form_async())
    print(result)
    return result

def google_site_search(query: str, num_results=3):
    url = "https://www.googleapis.com/customsearch/v1"
    params = {"q": query, "key": GOOGLE_API_KEY, "cx": CSE_ID, "num": num_results}
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json().get("items", [])
    return []

def scrape_text_from_url(url: str):
    try:
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.text, "html.parser")
        for tag in soup(["script", "style", "noscript"]): tag.decompose()
        clean_text = " ".join(soup.stripped_strings)
        return Document(page_content=clean_text[:4000], metadata={"source": url})
    except Exception as e:
        print(f"Error scraping {url}: {e}")
        return None

@tool
def dist(origin: str, destination: str) -> str:
    """
    Use this tool to find the driving distance and time between two locations.
    Provide a starting location (origin) and an ending location (destination).
    """
    print(f"⚙️ TOOL CALLED: calculate_distance from '{origin}' to '{destination}'")
    api_key = os.environ.get("GOOGLE_MAPS_API_KEY")
    if not api_key:
        return "Error: Google Maps API key is not configured."
    gmaps = googlemaps.Client(key=api_key)
    try:
        directions_result = gmaps.directions(origin, destination, mode="driving")
        if not directions_result:
            return f"Sorry, a route could not be found between {origin} and {destination}."
        leg = directions_result[0]['legs'][0]
        distance = leg['distance']['text']
        duration = leg['duration']['text']
        return f"The distance between {origin} and {destination} is {distance}. The estimated driving time is {duration}."
    except Exception as e:
        return f"An error occurred while fetching directions: {e}"

@tool
def search(query: str):
    """
    Use this tool to find information to answer general questions about the university.
    It searches online, extracts the most relevant information, and returns that specific context.
    """
    print(f"⚙️ TOOL CALLED: search with query: '{query}'")
    print("🔍 Performing Google CSE Search...")
    results = google_site_search(query)
    if not results:
        return "No relevant information found online. Inform the user you couldn't find anything."
    print("📄 Scraping and Retrieving Context...")
    links = [r["link"] for r in results if r.get("link")]
    if not links:
        return "Found search results, but no valid links. Inform the user."
    scraped_text = scrape_text_from_url(links[0])
    if not scraped_text or not scraped_text.page_content:
        return "Found a web page, but could not extract useful content. Inform the user."
    print("🧠 Extracting specific answer from the context...")
    extraction_prompt = f"""
    From the following text, please extract the exact answer to the question: "{query}"
    Text:
    ---
    {scraped_text.page_content}
    ---
    Provide only the specific information that answers the question. If the text does not contain the answer, state that.
    """
    extracted_answer = llm.invoke(extraction_prompt).content
    print(f"✅ Specific context extracted for the agent to use.")
    return extracted_answer





In [7]:
# --- Agent Setup (Your existing prompt and agent creation) ---
tools = [book_study_room, search, draft_email, send_email, dist]
prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        # Your existing system prompt here...
        """You are a helpful and efficient university assistant for the University of Texas at Arlington also called as UTA. You have four primary capabilities and after performing any of the below task you should be ready again to perform all task:

        1.  **Answering Questions:** For any general questions about the university of Texas at Arlington, you must use the `search` tool.

        2. **Booking Study Rooms**: When a user asks to book a study room, you must use the book_study_room tool. This tool requires a name, a date, a time, and the number of people. You must not guess or make up any of these details. If the user's initial request is missing any of this information, your only response should be to ask for the specific details you are missing. Do not call the tool until you have all four pieces of information from the user.

        3.  **Sending Emails:** This is a two-step process.
            - **Step 1: Draft the Email.** When a user asks to send an email, you **must** first use the `draft_email` tool. This tool requires the **recipient's email address**, a **subject line**, and the **email body**. If you don't have all three, you must ask the user for the missing information. The tool will show the draft to the user for approval.
            - **Step 2: Send the Email.** Only after the user confirms the draft by responding with 'Yes' or 'send it', you must then use the `send_email` tool with the exact same recipient, subject, and body. Do not ask for the information again. And send the email only once per request.

        4.  **Calculating Distances:** When a user asks for the distance or travel time between two places, use the `dist` tool. You must provide both an origin and a destination.

        Your goal is to be helpful and complete the user's request correctly by choosing the right tool and gathering all necessary information."""
    ),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)
print("✅ Agent and Executor Initialized.")

✅ Agent and Executor Initialized.


In [ ]:
def respond(message, history):
    chat_history = ChatMessageHistory()
    for human_message, ai_message in history:
        chat_history.add_message(HumanMessage(content=human_message))
        if ai_message is not None:
            chat_history.add_message(AIMessage(content=ai_message))

    response = agent_executor.invoke({
        "input": message,
        "chat_history": chat_history.messages,
    })
    return response["output"]

In [8]:
def transcribe_audio(audio_path):
    if not audio_path:
        return ""
    print(f"🎤 Transcribing audio: {audio_path}")
    transcribed_text = whisper_model.transcribe(audio_path)["text"]
    print(f"✅ Transcription complete: {transcribed_text}")
    return transcribed_text

In [9]:
# This function handles the chat logic when the "Submit" button is clicked
def chat_responder(message, history):
    """
    Takes the user's text message and chat history, gets the agent's response,
    updates the history, and clears the input box.
    """
    if not message:
        return history, "" # Do nothing if the input is empty

    # 1. Add the user's message to the chat history for display
    history.append((message, None))

    # 2. Call your original 'respond' function to get the bot's response
    bot_response = respond(message, history[:-1])

    # 3. Update the last entry in the history with the bot's full response
    history[-1] = (message, bot_response)

    # 4. Return the updated history and an empty string to clear the textbox
    return history, ""

In [10]:
# Your custom CSS from your old project
custom_css = """
body {
    margin: 0;
    padding: 0;
}
.custom-bg {
    background-image: url('https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse3.mm.bing.net%2Fth%2Fid%2FOIP.ndUCWbKmCM8W4iG3vkqWXwHaFR%3Fcb%3D12%26pid%3DApi&f=1&ipt=3d6430e1ee93a504b21a30a00bcd07d9a9bac80680b50da6b29e34a8a20838ad&ipo=images');
    background-size: cover;
    background-position: center;
    padding: 40px 0;
}
.glass-card {
    background: rgba(255, 255, 255, 0.2);
    backdrop-filter: blur(10px);
    padding: 40px;
    border-radius: 20px;
    width: 90%;
    max-width: 600px; /* Increased max-width slightly for the chat interface */
    margin: auto;
    box-shadow: 0 8px 32px rgba(0,0,0,0.3);
}
#mytextbox {
    background-color: rgba(204, 204, 204, 0.7) !important; /* Made it slightly transparent */
    padding: 12px !important;
    border-radius: 10px !important;
    border: 1px solid rgba(255, 255, 255, 0.3) !important;
}
#mytextbox textarea, #mytextbox input {
    background: transparent !important;
    border: none !important;
    resize: none !important;
    color: #000 !important;
    font-size: 16px !important;
    line-height: 1.4 !important;
}
#mytextbox textarea::placeholder, #mytextbox input::placeholder {
    color: #555 !important;
}
#mytextbox label {
    display: block;
    margin-bottom: 6px;
    color: #fff !important;
    font-weight: 500;
}
"""

with gr.Blocks(css=custom_css, theme=gr.themes.Soft()) as demo:
    with gr.Column(elem_classes="custom-bg"):
        with gr.Column(elem_classes="glass-card"):
            gr.Markdown("## <div style='text-align: center; color: white;'>UTA University Assistant 🎓</div>")

            # The chatbot component replaces your old output textbox
            chatbot = gr.Chatbot(label="Conversation", height=400)

            # The input textbox with the custom ID for styling
            input_text = gr.Textbox(
                label="Type your message or transcribe from audio below",
                elem_id="mytextbox"
            )
            submit_btn = gr.Button("Submit")

            gr.Markdown("<h3 style='text-align: center; color: white;'>------------------ Or ------------------</h3>")

            audio_input = gr.Audio(sources="microphone", type="filepath", label="Speak instead")
            transcribe_btn = gr.Button("🎙️ Transcribe to Textbox Above")

            # --- Wire the components to the functions ---

            # When "Transcribe" is clicked, the audio is processed and populates the input_text box
            transcribe_btn.click(
                fn=transcribe_audio,
                inputs=audio_input,
                outputs=input_text
            )

            # When "Submit" is clicked, the chat_responder function is called
            submit_btn.click(
                fn=chat_responder,
                inputs=[input_text, chatbot],
                outputs=[chatbot, input_text] # Outputs update the chatbot and clear the textbox
            )

demo.launch(debug=True, share=True)

/tmp/ipython-input-2216414628.py:48: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(css=custom_css, theme=gr.themes.Soft()) as demo:
/tmp/ipython-input-2216414628.py:48: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(css=custom_css, theme=gr.themes.Soft()) as demo:
/tmp/ipython-input-2216414628.py:54: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Conversation", height=400)
/tmp/ipython-input-2216414628.py:54: DeprecationWarning: The default value of 'allow_

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://035d58d1fe5843e928.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://035d58d1fe5843e928.gradio.live
